# Extraction of Laws References from the quiz questions

In [25]:
import pandas as pd
import json
import os
import re

isLinux = False

default_linux_path = os.getcwd() + "/work/"
default_windows_path = os.getcwd().replace("\\Data", "\\Documents\\Downloaded\\")
default_path = default_linux_path if isLinux else default_windows_path
DEFAULT_SAVE_DIR = default_path.replace("\\Downloaded", "\\Generated")

df = pd.read_csv(DEFAULT_SAVE_DIR + 'quiz_merged.csv')

df.head()

,Index,Question,Answer 1,Answer 2,Answer 3
0,1,Il Capo II della l. n. 241/1990 è riservato al...,"Accerta d'ufficio i fatti, disponendo il compi...",Non è mai competente alla valutazione della su...,É solo competente all'indizione delle conferen...
1,2,Ai sensi dell'art. 80 C.p.a. come avviene la p...,Deve essere presentata istanza di fissazione d...,Deve essere presentata istanza di fissazione d...,Deve essere presentato nuovamente il ricorso e...
2,3,Entro quale termine le parti devono proporre r...,Nel termine di sessanta giorni decorrente dall...,Nel termine di novanta giorni decorrente dalla...,Nel termine di centoventi giorni decorrente da...
3,4,A norma di quanto dispone l'art. 133 del C.p.a...,Alla giurisdizione esclusiva del giudice ammin...,Alla giurisdizione esclusiva del giudice ordin...,Alla giurisdizione esclusiva del TAR del Lazio.
4,5,Consacrando a livello costituzionale i princip...,"Moneta, tutela del risparmio e mercati finanzi...",Tutela e sicurezza del lavoro.,"Produzione, trasporto e distribuzione nazional..."


## Extraction of Penal Code's References

In [26]:
extracted_data = []
no_match_data = []

for i, row in df.iterrows():
    question = row['Question']
    #match = re.search(r'(artt?\.)? (\d+-\w+|\d+) (bis )?c\.p\.[^p^a]', question)
    match = re.search(r'artt?\. (\d+)[\w\d\s\-]{0,13}c\.p\.[^p^a]', question)
    match2 = re.search(r"(?:articolo|art\.) (\d+) del codice penale", question)

    if match or match2:
        article = match.group(1) if match else match2.group(1)
        extracted_data.append({
            'Law': 'c.p.',
            'Article': article,
            # 'Comma': comma,  # No quiz with comma references
            # 'Publication year': year,  # No quiz with year references
            'Quiz id': 1,
            'Question id': i + 1,
            # 'Link': link  # No quiz with link references
        })
    else:
        no_match_data.append(row.to_dict())

df_cp = pd.DataFrame(extracted_data)
df = pd.DataFrame(no_match_data)

print("CP found: ", df_cp.shape)
print("Still unmatched: ", df.shape)
df_cp.head()

CP found:  (136, 4)
Still unmatched:  (957, 5)


,Law,Article,Quiz id,Question id
0,c.p.,240,1,224
1,c.p.,266,1,225
2,c.p.,24,1,226
3,c.p.,7,1,227
4,c.p.,19,1,228


## Extraction of Administrative Penal Code's References

In [27]:
extracted_data = []
no_match_data = []

for i, row in df.iterrows():
    question = row['Question']
    #match = re.search(r'(artt?\.)? (\d+-\w+|\d+) (bis )?c\.p\.[^p^a]', question)
    match = re.search(r'artt?\. (\d+)[\w\d\s\-]{0,13}[Cc]\.p\.a[\.]?', question)

    if match:
        article = match.group(1)        
        extracted_data.append({
            'Law': 'c.p.a.',
            'Article': article,
            # 'Comma': comma,  # No quiz with comma references
            # 'Publication year': year,  # No quiz with year references
            'Quiz id': 1,
            'Question id': i + 1,
            # 'Link': link  # No quiz with link references
        })
    else:
        no_match_data.append(row.to_dict())

df_cpa = pd.DataFrame(extracted_data)
df = pd.DataFrame(no_match_data)

print("CPA found: ", df_cpa.shape)
print("Still unmatched: ", df.shape)
df_cpa.head()

CPA found:  (50, 4)
Still unmatched:  (907, 5)


,Law,Article,Quiz id,Question id
0,c.p.a.,80,1,2
1,c.p.a.,42,1,3
2,c.p.a.,133,1,4
3,c.p.a.,7,1,9
4,c.p.a.,34,1,10


## Extraction of Legislative Decree's References

## Check Missing Rows

In [28]:
# Print Question column of elements that were not matched
for i, row in df.iterrows():
    print(row['Question'])

Il Capo II della l. n. 241/1990 è riservato alla regolamentazione della figura del responsabile del procedimento, ovvero del soggetto al quale è affidato il delicato ruolo di autorità di guida di ciascun procedimento amministrativo. Esso: 
Consacrando a livello costituzionale i principi di cui alla L. 59/1997, in quale delle seguenti materie l'art. 117 della Costituzione attribuisce allo Stato potestà legislativa esclusiva? 
L'art. 5 del D.Lgs. n. 33/2013, dispone che il procedimento di accesso civico deve concludersi con provvedimento espresso e motivato nel termine di: 
Nel procedimento amministrativo, cosa prevede la Legge 241/1990, per come recentemente novellata dalla legge 69/2009, in caso di decorrenza del previsto termine senza che sia stato comunicato il parere obbligatorio di un organo consultivo o senza che esso abbia rappresentato esigenze istruttorie? 
A norma del disposto di cui all'art. 9, L. 400/1988, chi può conferire al Presidente del Consiglio dei ministri l'incarico